# Predicting Sports Winners with Decision Trees

Here, we will look at predicting the winner of sports matches using a different type of classification algorithm: decision trees. These algorithms have a number of advantages over other algorithms. One of the main advantages is that they are readable by humans. In this way, decision trees can be used to learn a procedure,
which could then be given to a human to perform if needed. Another advantage is that they work with a variety of features.

## Loading the dataset

We'll be looking at predicting the winner of games of the **National Basketball Association (NBA)**. Matches in the NBA are often close and can be decided in the last minute, making predicting the winner quite difficult. Many sports share this characteristic, whereby the expected winner could be beaten by another team on the right day.

Various research into predicting the winner suggests that there may be an upper limit to sports outcome prediction accuracy which, depending on the sport, is between 70 percent and 80 percent accuracy. There is a significant amount of research being performed into sports prediction, often through data mining or statistics-based methods.

## Collecting the data

The data we will be using is the match history data for the NBA for the 2013-2014 season. The website http://Basketball-Reference.com contains a significant number of resources and statistics collected from the NBA and other leagues. To download the dataset, perform the following steps:

1. Navigate to http://www.basketball-reference.com/leagues/NBA_2014_games.html in your web browser.
2. Choose to get table as csv file for each month
3. copy and paste the csv to your data folder and make a note of the path.

This will download a **CSV** (short for **Comma Separated Values**) file containing the
results of the 1,230 games in the regular season for the NBA.

CSV files are simply text files where each line contains a new row and each value is separated by a comma (hence the name). CSV files can be created manually by simply typing into a text editor and saving with a *.csv* extension. They can also be opened in any program that can read text files, but can also be opened in Excel as a spreadsheet.

We will load the file with the **pandas** (short for **Python Data Analysis**) library, which is an incredibly useful library for manipulating data. Python also contains a,built-in library called *csv* that supports reading and writing CSV files. However, we will use pandas, which provides more powerful functions that we will use later in the chapter for creating new features.

In [12]:
DATA = 'data/'
NBA_2014_SEASON = DATA + 'leagues_NBA_2014_games_games.csv'
NBA_EXTENDED_STANDINGS = DATA + 'leagues_NBA_2013_standings_expanded-standings.csv'

In [2]:
import numpy as np
import pandas as pd

# This is the entire season! Make sure to filter regular season
results = pd.read_csv(NBA_2014_SEASON)
results.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,LOG,Arena,Notes
0,Tue Oct 29 2013,7:00p,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,2:17,Bankers Life Fieldhouse,NaN
1,Tue Oct 29 2013,10:30p,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,2:27,STAPLES Center,NaN
2,Tue Oct 29 2013,8:00p,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,2:32,AmericanAirlines Arena,NaN
3,Wed Oct 30 2013,7:00p,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,2:23,Quicken Loans Arena,NaN
4,Wed Oct 30 2013,8:30p,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,2:14,American Airlines Center,NaN


## Cleaning up the dataset

After looking at the output, we can see a number of problems:
- The date is just a string and not a date object
- From visually inspecting the results, the headings aren't complete or correct

These issues come from the data, and we could fix this by altering the data itself. However, in doing this, we could forget the steps we took or misapply them; that is, we can't replicate our results. As with the previous section where we used pipelines to track the transformations we made to a dataset, we will use pandas to apply
transformations to the raw data itself.

In [11]:

# Parse the date column as a date
results = pd.read_csv(NBA_2014_SEASON, parse_dates=["Date"])
# Fix the name of the columns
results.columns = ['Date', 'Start (ET)', 'Visitor Team', 'VisitorPts', 'Home Team', 'HomePts', 'Score Type', 'OT?', 'Attend.', 'LOG', 'Arena', 'Notes']
results.head()

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,Score Type,OT?,Attend.,LOG,Arena,Notes
0,2013-10-29,7:00p,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,2:17,Bankers Life Fieldhouse,NaN
1,2013-10-29,10:30p,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,2:27,STAPLES Center,NaN
2,2013-10-29,8:00p,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,2:32,AmericanAirlines Arena,NaN
3,2013-10-30,7:00p,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,2:23,Quicken Loans Arena,NaN
4,2013-10-30,8:30p,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,2:14,American Airlines Center,NaN


Now that we have our dataset, we can compute a **baseline**. A baseline is an accuracy that indicates an easy way to get a good accuracy. Any data mining solution should beat this.

In each match, we have two teams: a home team and a visitor team. An obvious baseline, called the chance rate, is 50 percent. Choosing randomly will (over time) result in an accuracy of 50 percent.

## Extracting new features

We can now extract our features from this dataset by combining and comparing the existing data.

In [ ]:

results["HomeWin"] = results["VisitorPts"] < results["HomePts"]

# Our "class values"
y_true = results["HomeWin"].values
results.head()

In [ ]:
home_win_percentage = 100 * results["HomeWin"].sum() / results["HomeWin"].count()
print(f"Home Win percentage: {home_win_percentage:.1f}%")

In [ ]:
results["HomeLastWin"] = False
results["VisitorLastWin"] = False
# This creates two new columns, all set to False
results.head()

In [ ]:
# Now compute the actual values for these
# Did the home and visitor teams win their last game?
from collections import defaultdict
won_last = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    results.ix[index] = row    
    # Set current win
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]
results.ix[20:25]

In [ ]:

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [ ]:
from sklearn.model_selection import cross_val_score

# Create a dataset with just the neccessary information
X_previouswins = results[["HomeLastWin", "VisitorLastWin"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print("Using just the last result from the home and visitor teams")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
# What about win streaks?
results["HomeWinStreak"] = 0
results["VisitorWinStreak"] = 0
# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeWinStreak"] = win_streak[home_team]
    row["VisitorWinStreak"] = win_streak[visitor_team]
    results.ix[index] = row    
    # Set current win
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [ ]:
clf = DecisionTreeClassifier(random_state=14)
X_winstreak =  results[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values
scores = cross_val_score(clf, X_winstreak, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
# Let's try see which team is better on the ladder. Using the previous year's ladder
ladder = pd.read_csv(NBA_EXTENDED_STANDINGS, skiprows=[0,1])
ladder

In [ ]:

# We can create a new feature -- HomeTeamRanksHigher\
results["HomeTeamRanksHigher"] = 0
for index, row in results.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    if home_team == "New Orleans Pelicans":
        home_team = "New Orleans Hornets"
    elif visitor_team == "New Orleans Pelicans":
        visitor_team = "New Orleans Hornets"
    home_rank = ladder[ladder["Team"] == home_team]["Rk"].values[0]
    visitor_rank = ladder[ladder["Team"] == visitor_team]["Rk"].values[0]
    row["HomeTeamRanksHigher"] = int(home_rank > visitor_rank)
    results.ix[index] = row
results.head()

In [ ]:
X_homehigher =  results[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
from sklearn.model_selection import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_homehigher, y_true)
print(f"Accuracy: {grid.best_score_ * 100:.1f}%")

In [ ]:
# Who won the last match? We ignore home/visitor for this bit
last_match_winner = defaultdict(int)
results["HomeTeamWonLast"] = 0

for index, row in results.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    # Set in the row, who won the last encounter
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    results.ix[index] = row
    # Who won this one?
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner
results.head()

In [ ]:
X_home_higher =  results[["HomeTeamRanksHigher", "HomeTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_home_higher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(results["Home Team"].values)
home_teams = encoding.transform(results["Home Team"].values)
visitor_teams = encoding.transform(results["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Using full team labels is ranked higher")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:

X_all = np.hstack([X_home_higher, X_teams])
print(X_all.shape)

In [ ]:
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print(f"Accuracy: {np.mean(scores) * 100:.1f}%")

In [ ]:
#n_estimators=10, criterion='gini', max_depth=None, 
#min_samples_split=2, min_samples_leaf=1,
#max_features='auto',
#max_leaf_nodes=None, bootstrap=True,
#oob_score=False, n_jobs=1,
#random_state=None, verbose=0, min_density=None, compute_importances=None
parameter_space = {
                   "max_features": [2, 10, 'auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print(f"Accuracy: {grid.best_score_ * 100:.1f}%")
print(grid.best_estimator_)